# Linear Regression  on a Combined Cycle Power Plant (CCPP) data

## IMPORTANT: make sure to rerun all the code from the beginning to obtain the results for the final version of your notebook, since this is the way we will do it before evaluting your notebook!!!

## Dataset description

The dataset contains 9568 data points collected from a Combined Cycle Power Plant over 6 years (2006-2011), when the power plant was set to work with full load. Features consist of hourly average ambient variables Temperature (T), Ambient Pressure (AP), Relative Humidity (RH) and Exhaust Vacuum (V) to predict the net hourly electrical energy output (EP)  of the plant.

A combined cycle power plant (CCPP) is composed of gas turbines (GT), steam turbines (ST) and heat recovery steam generators. In a CCPP, the electricity is generated by gas and steam turbines, which are combined in one cycle, and is transferred from one turbine to another. While the Vacuum has effect on the Steam Turbine, the other three of the ambient variables effect the GT performance.

In [ ]:
from __future__ import division  
# to get in-line plots
%matplotlib nbagg
import matplotlib.pyplot as plt

import numpy as np
import scipy as sp
from scipy import stats

## Import Data
Load the data from a .csv file

** TO DO:** insert your ID number (matricola)


In [ ]:
# Load the data
IDnumber = 1110975
np.random.seed(IDnumber)

filename = "ccpp_Data_clean.csv"

Data = np.genfromtxt(filename, delimiter=';',skip_header=1)


# A quick overview of data

To inspect the data you can use the method describe()

In [ ]:
dataDescription = stats.describe(Data)
print(dataDescription)

Data.shape

#for more interesting visualization: use Panda!

# Split data in training, validation and test sets



Given $m$ total data, keep $m_t$ data as training data, $m_{val}:=m_{tv}-m_t$ as validation data and $m_{test}:=m - m_{val} - m_t = m-m_{tv}$. For instance one can take $m_t=m/3$ of the data as training, $m_{val}=m/3$  validation and $m_{test}=m/3$ as testing. Let us define as define

$\bullet$ $S_{t}$ the training data set

$\bullet$ $S_{val}$ the validation data set

$\bullet$ $S_{test}$ the testing data set


The reason for this splitting is as follows:

TRAINING DATA: The training data are used to compute the empirical loss
$$
L_S(h) = \frac{1}{m_t} \sum_{z_i \in S_{t}} \ell(h,z_i)
$$
which is used to estimate $h$ in a given model class ${\cal H}$.
i.e. 
$$
\hat{h} = {\rm arg\; min}_{h \in {\cal H}} \, L_S(h)
$$

VALIDATION DATA: When different model classes are present (e.g. of different complexity such as linear regression which uses a different number $d_j$ of regressors $x_1$,...$x_{d_j}$), one has to choose which one is the "best" complexity. 
Let ${\cal H}_{d_j}$ be the space of models as a function of the complexity $d_j$ and let 
$$
\hat h_{d_j} = {\rm arg\; min}_{h \in {\cal H}_{d_j}} \, L_S(h)
$$

One can estimate the generalization error for model $\hat h_{d_j}$ as follows:
$$
L_{{\cal D}}(\hat h_{d_j}) \simeq \frac{1}{m_{val}} \sum_{ z_i \in S_{val}} \ell(\hat h_{d_j},z_i)
$$
and then choose the complexity which achieves the best estimate of the generalization error
$$
\hat d_j: = {\rm arg\; min}_{d_j} \,\frac{1}{m_{val}} \sum_{ z_i \in S_{val}} \ell(\hat h_{d_j},z_i)
$$

TESTING DATA: Last, the test data set can be used to estimate the performance of the final estimated model
$\hat h_{\hat d_j}$ using:
$$
L_{{\cal D}}(\hat h_{\hat d_j}) \simeq \frac{1}{m_{test}} \sum_{ z_i \in S_{test}} \ell(\hat h_{\hat d_j},z_i)
$$


** TO DO **: split the data in training, validation and test sets (suggestion: use $m_t=m_{val} = \lfloor\frac{m}{3}\rfloor$, $m_{test} = m-m_t-m_{val}$)

In [ ]:
#get number of total samples
num_total_samples = Data.shape[0]

print "Total number of samples: ", num_total_samples

#size of each chunk of data for training, validation, testing
size_chunk = np.floor(num_total_samples/3).astype(int)
print "Size of each chunk of data: ", size_chunk

#shuffle the data
np.random.shuffle(Data)


#training data  

X_training = Data[:size_chunk, 0:4]
Y_training = Data[:size_chunk, 4:]
print "Training input data size: ", X_training.shape
print "Training output data size: ", Y_training.shape

#validation data, to be used to choose among different models
X_validation = Data[size_chunk:2*size_chunk, 0:4]
Y_validation = Data[size_chunk:2*size_chunk, 4:]
print "Validation input data size: ", X_validation.shape
print "Validation output data size: ", Y_validation.shape

#test data, to be used to estimate the true loss of the final model(s)
X_test = Data[2*size_chunk:, 0:4]
Y_test = Data[2*size_chunk:, 4:]
print "Test input data size: ", X_test.shape
print "Test output data size: ", Y_test.shape

# Data Normalization

It is common practice in Statistics and Machine Learning to scale the data (= each variable) so that it is centered (zero mean) and has standard deviation equal to $1$. This helps in terms of numerical conditioning of the (inverse) problems of estimating the model (the coefficients of the linear regression in this case), as well as to give the same scale to all the coefficients. 

In [ ]:
#scale the data

# standardize the input matrix
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_training)
X_training = scaler.transform(X_training)
print "Mean of the training input data:", X_training.mean(axis=0)
print "Std of the training input data:",X_training.std(axis=0)
X_validation = scaler.transform(X_validation) # use the same transformation on validation data
print "Mean of the validation input data:", X_validation.mean(axis=0)
print "Std of the validation input data:", X_validation.std(axis=0)
X_test = scaler.transform(X_test) # use the same transformation on test data
print "Mean of the test input data:", X_test.mean(axis=0)
print "Std of the test input data:", X_test.std(axis=0)



# Model Training 

The model is trained (= estimated) minimizing the empirical error
$$
L_S(h) := \frac{1}{m_t} \sum_{z_i \in S_{t}} \ell(h,z_i)
$$
When the loss function is the quadratic loss
$$
\ell(h,z) := (y - h(x))^2
$$
we define  the Residual Sum of Squares (RSS) as
$$
RSS(h):= \sum_{z_i \in S_{t}} \ell(h,z_i) = \sum_{z_i \in S_{t}} (y_i - h(x_i))^2
$$ so that the training error becomes
$$
L_S(h) = \frac{RSS(h)}{m_t}
$$

We recal that, for linear models we have $h(x) = <w,x>$ and the Empirical error $L_S(h)$ can be written
in terms of the vector of parameters $w$ in the form
$$
L_S(w) = \frac{1}{m_t} \|Y - X w\|^2
$$
where $Y$ and $X$ are the matrices whose $i-$th row are, respectively, the output data $y_i$ and the input vectors $x_i^\top$.

The least squares solution is given by the expression
$$
\hat w = {\rm arg\;min}_w L_S(w) = (X^\top X)^{-1} X^\top Y
$$
When the matrix $X$ is not invertible, the solution can be computed using the Moore-Penrose pseudonverse $(X^\top X)^{\dagger}$ of $(X^\top X)$
$$
\hat w = (X^\top X)^{\dagger} X^\top Y
$$
The Moore-Penrose pseudoinverse $A^\dagger$ of a matrix $A \in \mathbb{R}^{m\times n}$ can be expressed in terms of the Singular Value Decomposition (SVD) as follows:
Let $A\in \mathbb{R}^{m\times n}$ be of rank $r\leq {\rm min}(n,m)$ and let  
$$
 A = USV^\top
 $$
 be the singular value decomposition of  $A$ where  
 $$
 S = {\rm diag}\{s_1,s_2,..,s_r\}
 $$
 Then 
 $$
 A^\dagger =V S^{-1} U^\top 
 $$
 
 In practice some of the singular values may be very small (e.g. $<1e-12$). Therefore it makes sense to 
 first approximate the matrix $A$ truncating the SVD and then using the pseudoinverse formula.
 
 More specifically, let us postulate that, given a threshold $T_h$ (e.g $=1e-12$), we have $s_i<T_h$, for $i=\hat r + 1,..,r$. Then we can approximate (by SVD truncation) $A$ using:
 
 $$A = USV^\top =U \,{\rm diag}\{s_1,s_2,..,s_r\}\, V^\top \simeq \hat A_r = U\,{\rm diag}\{s_1,s_2,..,s_{\hat r}, 0,..,0\}\,V^\top
 $$
 So that 
 $$
 A^\dagger \simeq \hat A_r^\dagger:= V \,{\rm diag}\{1/s_1,1/s_2,..,1/s_{\hat r}, 0,..,0\}\, U^\top
 $$
  
 ** TO DO **: compute the linear regression coefficients directly and using np.linalg.lstsq from scikitlear 

In [ ]:
#compute linear regression coefficients for training data

#add a 1 at the beginning of each sample for training, validation, and testing
m_training = X_training.shape[0]

X_training = np.hstack((np.ones((m_training,1)),X_training))
#print X_training[0,:]

m_validation = X_validation.shape[0]
X_validation = np.hstack((np.ones((m_validation,1)),X_validation))
#print X_validation[0,:]

m_test = X_test.shape[0]
X_test = np.hstack((np.ones((m_test,1)),X_test))
#print X_test[0,:]

# Compute the least-squares solution using SVD AND PSEUDOINVERSE AS ABOVE
pA = np.linalg.pinv(np.dot(np.transpose(X_training), X_training), rcond=1e-12)
w_hand = np.dot(pA, np.dot(np.transpose(X_training), Y_training))

# Compute the least-squares coefficients using linalg.lstsq
A = np.dot(np.transpose(X_training), X_training)
b = np.dot(np.transpose(X_training), Y_training)
w_np, RSStr_np, rank_Xtr, sv_Xtr = np.linalg.lstsq(A, b, rcond=1e-12) # if A is a (N,M) matrix and the rank of A is n<M 
                                                                      # or M<=N then this is an empty array

print "LS coefficients by hand:", w_hand

print "LS coefficients with numpy lstsq:", w_np

# compute Residual sums of squares by hand
RSStr_hand = 0.
for i in range(X_training.shape[0]):
    h = np.inner(np.transpose(w_hand), np.transpose(X_training[i]))
    RSStr_hand += np.power(np.subtract(Y_training[i], h), 2)

print "RSS by hand:",  RSStr_hand
print "RSS with numpy lstsq: ", RSStr_np # if A is a (N,M) matrix and the rank of A is n<M 
                                         # or M<=N then this is an empty array

print "Empirical risk by hand:", RSStr_hand/m_training
print "Empirical risk with numpy lstsq:", RSStr_np/m_training



** COMMENT **: The RSS found by the np.linalg.lstsq procedure is an empty array. In the documentation of that procedure it can be read that if $A$ is a $(N,M)$ matrix and the rank of A is $n<M$ or $M\leq N$ then this is an empty array. 

## Data prediction 

Compute the output predictions on both training and validation set and compute the Residual Sum of Sqaures (RSS) defined above, the Emprical Loss and the quantity $1-R^2$ where
$$
R^2 = \frac{\sum_{z_i \in S_t} (\hat y_i - \bar y)^2}{\sum_{z_i \in S_t} (y_i - \bar y)^2} \quad \quad \bar y = \frac{1}{m_t} \sum_{z_i \in S_t} y_i
$$
is the so-called "Coefficient of determination" (COD)

** TO DO**: Compute these quantities on  training, validation and test sets.


In [ ]:
#compute predictions on training and validation
prediction_training = np.dot(X_training, w_hand)
prediction_validation = np.dot(X_validation, w_hand)
prediction_test = np.dot(X_test, w_hand)

#what about the loss for points in the validation data?
RSS_validation = 0.
for i in range(X_validation.shape[0]):
    RSS_validation += np.power(np.subtract(Y_validation[i], prediction_validation[i]), 2)
    
RSS_test = 0.
for i in range(X_test.shape[0]):
    RSS_test += np.power(np.subtract(Y_test[i], prediction_test[i]), 2)

print "RSS on validation data:",  RSS_validation
print "Loss estimated from validation data:", RSS_validation/m_validation


#another measure of how good our linear fit is given by the following (that is 1 - R^2)
y_bar_training = np.sum(Y_training) / m_training
numerator_training = np.sum(np.power(np.subtract(prediction_training, np.full(prediction_training.shape, y_bar_training)) , 2))
denominator_training = np.sum(np.power(np.subtract(Y_training, np.full(Y_training.shape, y_bar_training)) , 2))
Rmeasure_training = 1 - numerator_training / denominator_training

y_bar_validation = np.sum(Y_validation) / m_validation
numerator_validation = np.sum(np.power(np.subtract(prediction_validation, np.full(prediction_validation.shape, y_bar_validation)) , 2))
denominator_validation = np.sum(np.power(np.subtract(Y_validation, np.full(prediction_validation.shape, y_bar_validation)) , 2))
Rmeasure_validation = 1 - numerator_validation / denominator_validation 

y_bar_test = np.sum(Y_test) / m_test
numerator_test = np.sum(np.power(np.subtract(prediction_test, np.full(prediction_test.shape, y_bar_test)) , 2))
denominator_test = np.sum(np.power(np.subtract(Y_test, np.full(prediction_test.shape, y_bar_test)) , 2))
Rmeasure_test = 1 - numerator_test / denominator_test 

print "Measure on Training Data (1-R^2):", Rmeasure_training
print "Measure on Validation Data(1-R^2):", Rmeasure_validation
print "Measure on Test Data(1-R^2):", Rmeasure_test

## ... and plot:


### (1) output predictions on training  data

In [ ]:
# Plot predictions on Training data 
plt.figure()

#the following is just for nice plotting, not required: it sorts the predictions by value so that they fall on
# a line and it's easier to spot the differences
sorting_permutation = sorted(range(len(prediction_training[0:m_training])), key=lambda k: prediction_training[0:m_training][k])
plt.plot(Y_training[sorting_permutation], 'ko', alpha=0.5)
plt.plot(prediction_training[sorting_permutation], 'rx')

plt.xlabel('Input (index of instance)')
plt.ylabel('Predicted Output')
plt.title('Predictions on Training Data')
plt.show()

### (2) output predictions on validation  data

In [ ]:
# Plot predictions on validation data 
plt.figure()

#the following is just for nice plotting, not required: it sorts the predictions by value so that they fall on
# a line and it's easier to spot the differences
sorting_permutation = sorted(range(len(prediction_validation[0:m_validation])), key=lambda k: prediction_validation[0:m_validation][k])
plt.plot(Y_validation[sorting_permutation], 'ko', alpha=0.5)
plt.plot(prediction_validation[sorting_permutation], 'gx')


plt.xlabel('Input (index of instance)')
plt.ylabel('Predicted Output')
plt.title('Predictions on Validation Data')
plt.show()

## Hypothesis testing:

Test significance of each LS coefficient: find rejection regions for the null hypothesis

$$
H_0 : w_j =0
$$

The rejection region (with type-1 probability of error $\alpha$) has the form

$$
{\cal R}_j:=\left\{|\hat w_j|> \Delta\right\} \quad \quad \Delta: = \hat\sigma\ z_{jj}\  t_{1-\frac{\alpha}{2}}(m_t-d-1)
$$
where
$$
\hat\sigma^2 : = \frac{1}{m_t-d-1} RSS(\hat w)
$$
is the estimator of the noise variance, $z_{jj}$ is the square root of the $j-th$ diagonal element of $\left(X^\top X\right)^{-1}$ (here $X$ contains only the training data) and $t_{1-\frac{\alpha}{2}}(m_t-d-1)$ is the $1-\frac{\alpha}{2}$-percentile of the $T$ distribution with $m_t-d-1$ degrees of freedom ($d=4$ in our case), i.e. 
$$
\mathbb{P}[T(m_t-d-1)\leq t_{1-\frac{\alpha}{2}}(m_t-d-1)] = 1-\frac{\alpha}{2}
$$

The complement of the rejection region is also called the acceptance region (i.e. the set of data where the null hypothesis $H_0$ is accepted). In particular:

$$
{\cal A}_j:={\cal R}_j^c = \left\{|\hat w_j|\leq  \Delta\right\} \quad \quad \Delta: = \hat\sigma\ z_{jj}\  t_{1-\frac{\alpha}{2}}(m_t-d-1)
$$

**TO DO:** compute the acceptance region for all coefficients

In [ ]:
#compute confidence interval for regression coefficients

# fix type-1 error probability and compute percentiles

from scipy.stats import t

alpha = 0.05 # FIXED CONFIDENCE
d = X_training.shape[1] - 1

print d

t_percentile = t.ppf(1-alpha/2, m_training - d - 1 , loc=0, scale=1) # COMPUTE 1-alpha/2 PERCENTILE of the T distribution

# Estimate noise variance
sigma2 = RSStr_hand / (m_training - d - 1)

#compute the term to be added and subtracted to obtain the CI's
inv = np.linalg.inv(np.dot(np.transpose(X_training), X_training))
Delta = np.zeros(X_training.shape[1])
for i in range(Delta.shape[0]):
    Delta[i] = np.sqrt(sigma2) * np.sqrt(inv[i,i]) * t_percentile

print Delta

A = np.zeros((X_training.shape[1],2))
for i in range(A.shape[0]):
    upper = w_hand[i] + Delta[i]
    lower =  w_hand[i] - Delta[i]
    if (lower <= upper):
        A[i,0] = lower
        A[i,1] = upper
    else:
        A[i,0] = upper
        A[i,1] = lower

## Inspect Acceptance Regions
Now let's have a look at the acceptance regions. The $j-th$ row of the matrix $A$ contains the lower and upper intervals (respectively) of the acceptance region for the test above on the $j-th$ coefficient $w_j$. If $\hat w_j$ belongs to the acceptance region we can accept the hypothesis that the $j-th$ regressor is not relevant. 

In [ ]:
print "LS coefficients:",  w_hand
print "Acceptance Regions:", A

Equivalently, we can translate by a quantity $\hat w_j$ the Acceptance region and accept the hypothesis $H_0$ if the translated interval contains the origin 



In [ ]:
CI = w_hand + A

print "Confidence Intervals:", CI

Note that the intervals whose extremes are in $CI$ are nothing but the confidence intervals for $w$!

## Now plot confidence intervals (= translated acceptance regions!!!)

In [ ]:
# Plot confidence intervals for the coeffients
fig=plt.figure()
axx=fig.add_subplot(111)
axx.plot(w_hand, 'r', marker='o', ms=10.0)
axx.fill_between(range(len(w_hand)), CI[:,0], CI[:,1], alpha=0.5)
axx.plot(np.zeros(w_hand.shape[0],), 'k', linewidth=2.0)
plt.xlabel('Coefficient Index')
plt.ylabel('LS Coefficient')
plt.title('Coefficients and Confidence Sets')
plt.show()

# Remove useless regressors

Perform the learning as above removing not useful regressors; recall that this can be done using hypothesis testing. On the full model one can test, for each coefficient, whether the null hypothesis that the coefficient is zero should be accepted or rejected. If this procedure yields to no acceptance  (that $w_j=0$ for some $j$) choose the most reasonable coefficient to be removed from the regressor vector.


**WARNING:** You should be careful if the null hypothesis that $w_j=0$ is accepted for more than one $j$ (i.e. one is tempted to remove more than one coefficient from the regression). We shall discuss this issue in class later on.

**TO DO**: repeat the learning procedure after having removed non useful regressors and explain the choice of which regressors to be removed.

In [ ]:
# reduced design matrix

# Since the null hypothesis testing does not give us any meaningfull result, I decide to remove those regressors
# that have coefficients very close to zero, such as regressors of indexes 3 and 4.
selected_regressors = [0, 1, 2]
X_training_reduced = X_training[:,selected_regressors]

X_validation_reduced = X_validation[:,selected_regressors]

A_reduced = np.dot(np.transpose(X_training_reduced), X_training_reduced)
b_reduced = np.dot(np.transpose(X_training_reduced), Y_training)
w_np_reduced, RSStr_np_reduced, rank_Xtr_reduced, sv_Xtr_reduced = np.linalg.lstsq(A_reduced, b_reduced, rcond=1e-12)

print "LS coefficients:", w_np_reduced

# Compute predictions
prediction_training_reduced = np.dot(X_training_reduced, w_np_reduced)
prediction_validation_reduced = np.dot(X_validation_reduced, w_np_reduced)


# Compute Validation Error
RSS_validation_reduced = 0.
for i in range(X_training_reduced.shape[0]):
    RSS_validation_reduced += np.power(np.subtract(Y_validation[i], prediction_validation_reduced[i]), 2)

print "RSS on validation data:",  RSS_validation_reduced
print "Loss estimated from validation data:", RSS_validation_reduced/m_validation


#another measure of how good our linear fit is, is given by the following (that is 1 - R^2)
# Compute Training and Validation Error
y_bar_training_reduced = np.sum(Y_training) / m_training
numerator_training_reduced = np.sum(np.power(np.subtract(prediction_training_reduced, np.full(prediction_training_reduced.shape, y_bar_training_reduced)) , 2))
denominator_training_reduced = np.sum(np.power(np.subtract(Y_training, np.full(Y_training.shape, y_bar_training_reduced)) , 2))
Rmeasure_training_reduced = 1 - numerator_training_reduced / denominator_training_reduced

y_bar_validation_reduced = np.sum(Y_validation) / m_validation
numerator_validation_reduced = np.sum(np.power(np.subtract(prediction_validation_reduced, np.full(prediction_validation_reduced.shape, y_bar_validation_reduced)) , 2))
denominator_validation_reduced = np.sum(np.power(np.subtract(Y_validation, np.full(prediction_validation_reduced.shape, y_bar_validation_reduced)) , 2))
Rmeasure_validation_reduced = 1 - numerator_validation_reduced / denominator_validation_reduced 

print "Measure on Validation Data:", Rmeasure_validation_reduced

## Plot prediction on validation data reduced regression

In [ ]:
plt.figure()

#the following is just for nice plotting, not required: it sorts the predictions by value so that they fall on
# a line and it's easier to spot the differences
sorting_permutation = sorted(range(len(prediction_validation_reduced[0:m_validation])), key=lambda k: prediction_validation_reduced[0:m_validation][k])
plt.plot(Y_validation[sorting_permutation], 'ko', alpha=0.5)
plt.plot(prediction_validation_reduced[sorting_permutation], 'gx')


plt.xlabel('Input (index of instance)')
plt.ylabel('Predicted Output')
plt.title('Predictions on Validation Data')
plt.show()

# MODEL SELECTION 

** TO DO**: Based on the results of the estimated models (full model and reduced model) on the validation data which one would you choose? Explain the choice.

** ANSWER ** 
Based on the results above, I would choose the reduced model since the R measure is better than the one of the full model. Since the null hypothesis testing done before does not give indication to any particular regressors that could be removed, I chose arbitrarily to remove those regressors that have coefficients very close to zero: this selection is not particulary usefull since the R measures on both full and reduced model are very close to each other; nonetheless, the reduced model give us better estimates on the validation set.

## Visualization:  3D plot with fitted hyperplane (only w.r.t. 2 regressors) for training set

To the purpose of visualization we plot the linear function only w.r.t. the first 2 variables (of course any other choice could be made)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.gca(projection='3d')
#plot only m=100 points
m = 100
x1 = X_training_reduced[0:m-1,1]
x2 = X_training_reduced[0:m-1,2]
ax.scatter(x1,x2,prediction_training_reduced[0:m-1], c='r')

#generate a meshgrid to be used to plot surface

#n = number of points to be used in the grid
n=10
x1_surf = np.arange(0.9*min(x1), 1.1*max(x1), (1.1*max(x1)-0.9*min(x1))/n)
x2_surf = np.arange(0.9*min(x2), 1.1*max(x2), (1.1*max(x2)-0.9*min(x2))/n)                

# generate a mesh
x1_surf, x2_surf = np.meshgrid(x1_surf, x2_surf)

#get corresponding input and then the predicted output from linear regression model
x_surf = np.hstack((np.ones((n*n,1)),zip( x1_surf.ravel(),x2_surf.ravel()  )))
y_surf = np.dot(x_surf,w_np_reduced)

ax.plot_surface(x1_surf,x2_surf,y_surf.reshape(n,n), alpha=0.5, color=[1,1,1])

plt.show()

## Confidence intervals for output predictions

Having estimated the coefficients $\hat w$, and given a new location $x_0$, the output prediction  has the form
$$
\hat y_0 : = x_0 ^\top \hat w
$$
and postulating that 
$$
y_0 = x_0^\top w + \epsilon_0 \quad \epsilon_0 \sim {\cal N}(0,\sigma^2)
$$

we would like to compute a confidence interval on the output prediction or equivalently for the estimation error
$$
\tilde y_0:=\hat y_0 - y_0$$
Using the equations above we have that 
$$
\tilde y_0 =x_0^\top (\hat w - w) -  \epsilon_0
$$
where $\epsilon_0$ and $\hat w$ are uncorrelated. It thus follows that 
$$
\tilde y_0 \sim {\cal N}(0, x_0^\top {Var}\{\hat w\}x_0  + \sigma^2)
$$
where
$$
x_0^\top {Var}\{\hat w\}x_0 = \sigma^2 x_0^\top \left(X X^\top\right)^{-1}x_0 \quad \quad
X:=[x_1,..,x_m]
$$
using the results we have seen in class (extended to the case of unknown variance) we have that the interval 
$$
[ - \Delta_0, + \Delta_0] \quad \quad \Delta_0 : = \hat\sigma \, t_{1-\frac{\alpha}{2}}(m_{t}-d-1) \sqrt{x_0^\top (X^\top X)^{-1}x_0 + 1} 
$$
where 
$$
\hat\sigma^2:= \frac{1}{m_t-d-1}\sum_{i\in S_t} (y_i - \hat w^\top x_i)^2
$$
satisfies the condition
$$
\mathbb{P}[\tilde y_0 \in [ - \Delta_0, + \Delta_0]] =\mathbb{P}[\hat  y_0 - y_0 \in [ - \Delta_0, + \Delta_0]]= 1-\alpha
$$
Equivalently we can write
$$
\mathbb{P}[y_0 \in [ \hat y_0- \Delta_0,  \hat y_0 + \Delta_0]]  = 1-\alpha
$$
Note that in the latter equation both $y_0$ AND the interval $[ \hat y_0- \Delta_0,  \hat y_0 + \Delta_0]$ are random. The probability is to be understood with repect to both the choice of the traning set $(Y,X)$  as well as on the choice of $y_0$. 
With some abuse of terminology we shall say that $[ \hat y_0- \Delta_0,  \hat y_0 + \Delta_0]$ is a confidence interval of level $1-\alpha$ for $y_0$.


**TO DO**: compute confidence intervals for output prediction of FULL model

In [ ]:
# compute predictions on test data
# prediction_test is already computed for the FULL model
from scipy.stats import t
alpha_test = 0.05
d_test = X_test.shape[1] - 1
x_0 = X_test[np.random.randint(0, m_test),:] # choose x_0 randomly from the test set
t_percentile_test = t.ppf(1-alpha_test/2, m_test - d_test - 1 , loc=0, scale=1)
inv_term_test = np.linalg.inv(np.dot(np.transpose(X_test), X_test))
sqrt_term_test = np.sqrt(np.add(1, np.dot(x_0, np.dot(inv_term_test, np.transpose(x_0)))))
var_term_test = 0.
for i in range(X_test.shape[0]):
    var_term_test += (Y_test[i]-np.dot(np.transpose(w_hand), X_test[i]))**2
var_term_test /= (m_test - d_test - 1)

Delta_0 = np.sqrt(var_term_test) * t_percentile_test * sqrt_term_test

CI_0 = np.zeros((X_test.shape[0], 2))
for i in range(CI_0.shape[0]):
    lower = prediction_test[i] - Delta_0
    upper = prediction_test[i] + Delta_0
    if lower <= upper:
        CI_0[i,0] = lower
        CI_0[i,1] = upper
    else:
        CI_0[i,0] = upper
        CI_0[i,1] = lower

In [ ]:
fig=plt.figure(figsize = (12,10))
axx =fig.add_subplot(111)
#the following is just for nice plotting, not required: it sorts the predictions by value so that they fall on
# a line and it's easier to spot the differences
sorting_permutation = sorted(range(len(prediction_test[0:m_test])), key=lambda k: prediction_test[0:m_test][k])
axx.plot(Y_test[sorting_permutation], 'ko', alpha=0.5)
axx.plot(prediction_test[sorting_permutation], 'bx')
axx.fill_between(range(m_test), CI_0[sorting_permutation,0],CI_0[sorting_permutation,1], alpha=0.4)

plt.xlabel('Input (index of instance)')
plt.ylabel('Predicted Output')
plt.title('Predictions on Test Data')
plt.show()

## Ordinary Least-Squares using scikit-learn

A fast way to compute the LS estimate is through sklearn.linear_model

In [ ]:
# Remove the ``ones'' column in the features matrix (sklearn inserts it automatically)
X_training_sl = X_training[:,1:]
X_test_sl = X_test[:,1:]


In [ ]:
from sklearn import linear_model
LinReg = linear_model.LinearRegression()  # build the object LinearRegression
LinReg.fit(X_training_sl, Y_training)  # estimate the LS coefficients
print "Intercept:", LinReg.intercept_
print "Least-Squares Coefficients:", LinReg.coef_
prediction_training = LinReg.predict(X_training_sl)  # predict output values on training set
prediction_test = LinReg.predict(X_test_sl)  # predict output values on test set
print "Measure on training data:", 1-LinReg.score(X_training_sl, Y_training)
